In [39]:
# Load the extension that allows us to compile CUDA code in python notebooks
# Documentation is here: https://nvcc4jupyter.readthedocs.io/en/latest/
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc4jupyter



  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-9cgjep6z
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-9cgjep6z
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 28f872a2f99a1b201bcd0db14fdbc5a496b9bfd7
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
The nvcc4jupyter extension is already loaded. To reload it, use:
  %reload_ext nvcc4jupyter


# 新段落

In [40]:
import torchvision
import os

def download_mnist_dataset():
    # 创建目录
    os.makedirs("train_mnist/MNIST/raw", exist_ok=True)
    os.makedirs("test_mnist/MNIST/raw", exist_ok=True)

    # 下载训练数据
    train_dataset = torchvision.datasets.MNIST(root='./data', train=True, download=True)
    test_dataset = torchvision.datasets.MNIST(root='./data', train=False, download=True)

    print("MNIST dataset downloaded successfully.")

# 调用函数下载数据集
download_mnist_dataset()

MNIST dataset downloaded successfully.


In [41]:
'''DO NOT UNCOMMENT THIS CELL unless you are running this notebook on Google Colab'''
# from google.colab import drive
# drive.mount('/content/drive/', force_remount=True)


'DO NOT UNCOMMENT THIS CELL unless you are running this notebook on Google Colab'

In [42]:
%%cuda_group_save -g "knn" -n "main.cu"

// Required header files / 所需的头文件
#include <iostream>     // For input/output operations / 用于输入输出操作
#include <fstream>      // For file operations / 用于文件操作
#include <vector>       // For vector container / 用于向量容器
#include <string>       // For string operations / 用于字符串操作
#include <cstring>      // For C-style string operations / 用于C风格字符串操作
#include <algorithm>    // For algorithms like max_element / 用于算法如max_element
#include <cuda_runtime.h> // For CUDA operations / 用于CUDA操作
#include <cfloat>

// Constants definition / 常量定义
#define THREADS 256        // Number of threads per block / 每个块的线程数
#define IMAGESIZE 784      // Image size (28x28 = 784 pixels) / 图像大小 (28x28 = 784像素)
#define TILE_WIDTH 32
#define SHARED_MEM_SIZE (TILE_WIDTH * IMAGESIZE)
#define max_samples = 1000

// Function to handle big-endian to little-endian conversion
// 处理大端序转小端序的函数
uint32_t swap32(uint32_t val) {
    val = ((val << 8) & 0xFF00FF00) | ((val >> 8) & 0xFF00FF);
    return (val << 16) | (val >> 16);
}

// structure to store training/testing samples
// 存储训练/测试样本的结构体
struct TrainingSample {
    int label;                  // The digit (0-9) / 数字标签 (0-9)
    float image[IMAGESIZE];     // Normalized pixel values / 归一化的像素值
};

__global__ void bitonicSortStep(float* d_distances, int* d_labels, int j, int k, int num_samples) {
    // 使用共享内存来减少全局内存访问
    __shared__ float s_distances[1024];  // 设置为合适的大小
    __shared__ int s_labels[1024];

    const unsigned int tid = threadIdx.x;
    const unsigned int bid = blockIdx.x;
    const unsigned int gid = bid * blockDim.x + tid;
    const unsigned int gridSize = gridDim.x * blockDim.x;

    // 每个线程块处理多个元素
    for (unsigned int i = gid; i < num_samples; i += gridSize) {
        s_distances[tid] = d_distances[i];
        s_labels[tid] = d_labels[i];
        __syncthreads();

        unsigned int ixj = i ^ j;

        if (ixj > i && ixj < num_samples) {
            float dist_i = s_distances[tid];
            float dist_ixj = d_distances[ixj];
            int label_i = s_labels[tid];
            int label_ixj = d_labels[ixj];

            bool swap = ((i & k) == 0) ? (dist_i > dist_ixj) : (dist_i < dist_ixj);

            if (swap) {
                s_distances[tid] = dist_ixj;
                s_labels[tid] = label_ixj;
                d_distances[ixj] = dist_i;
                d_labels[ixj] = label_i;
            }
        }
        __syncthreads();

        d_distances[i] = s_distances[tid];
        d_labels[i] = s_labels[tid];
    }
}

void bitonicSort(float* d_distances, int* d_labels, int num_samples) {
    // 计算最近的2的幂
    int pow2_size = 1;
    while (pow2_size < num_samples) pow2_size <<= 1;
    int padded_size = pow2_size;

    // 一次性初始化填充数据
    if (padded_size > num_samples) {
        const float max_distance = FLT_MAX;
        const int max_label = -1;

        // 分配并初始化填充数据
        float* h_pad_distances = new float[padded_size - num_samples];
        int* h_pad_labels = new int[padded_size - num_samples];

        for (int i = 0; i < padded_size - num_samples; ++i) {
            h_pad_distances[i] = max_distance;
            h_pad_labels[i] = max_label;
        }

        cudaMemcpy(d_distances + num_samples, h_pad_distances,
                  (padded_size - num_samples) * sizeof(float), cudaMemcpyHostToDevice);
        cudaMemcpy(d_labels + num_samples, h_pad_labels,
                  (padded_size - num_samples) * sizeof(int), cudaMemcpyHostToDevice);

        delete[] h_pad_distances;
        delete[] h_pad_labels;
    }

    // 优化的网格配置
    const int threadsPerBlock = 256;
    const int numBlocks = (padded_size + threadsPerBlock - 1) / threadsPerBlock;

    // 使用循环展开来减少循环开销
    #pragma unroll 4
    for (int k = 2; k <= pow2_size; k <<= 1) {
        #pragma unroll 4
        for (int j = k >> 1; j > 0; j >>= 1) {
            bitonicSortStep<<<numBlocks, threadsPerBlock>>>(
                d_distances, d_labels, j, k, padded_size);
        }
    }
}


__global__ void computeEuclideanDistances(float* d_images, float* d_testImage,
                                          float* d_distances, int* d_labels,
                                          int* d_train_labels, int num_samples) {
    __shared__ float shared_test[IMAGESIZE];

    int tid = threadIdx.x;
    int bid = blockIdx.x;
    int idx = bid * blockDim.x + tid;

    // 高效加载测试图像到共享内存
    #pragma unroll
    for (int i = 0; i < IMAGESIZE; i += blockDim.x) {
        if (i + tid < IMAGESIZE) {
            shared_test[i + tid] = d_testImage[i + tid];
        }
    }
    __syncthreads();

    if (idx < num_samples) {
        float sum = 0.0f;

        // 使用更高效的计算方式
        #pragma unroll 32
        for (int i = 0; i < IMAGESIZE; ++i) {
            float diff = d_images[idx * IMAGESIZE + i] - shared_test[i];
            sum += diff * diff;
        }

        d_distances[idx] = sqrtf(sum);
        d_labels[idx] = d_train_labels[idx];
    }
}

// function to load MNIST dataset in IDX format
// 加载IDX格式MNIST数据集的函数
bool loadMNISTImages(const std::string& image_path, const std::string& label_path,
                    std::vector<TrainingSample>& samples) {
    // Open image file / 打开图像文件
    std::ifstream image_file(image_path, std::ios::binary);
    if (!image_file) {
        std::cerr << "Cannot open image file: " << image_path << std::endl;
        return false;
    }

    // Open label file / 打开标签文件
    std::ifstream label_file(label_path, std::ios::binary);
    if (!label_file) {
        std::cerr << "Cannot open label file: " << label_path << std::endl;
        return false;
    }

    // Read image file header / 读取图像文件头
    uint32_t magic, num_items, num_rows, num_cols;
    image_file.read(reinterpret_cast<char*>(&magic), sizeof(magic));
    image_file.read(reinterpret_cast<char*>(&num_items), sizeof(num_items));
    image_file.read(reinterpret_cast<char*>(&num_rows), sizeof(num_rows));
    image_file.read(reinterpret_cast<char*>(&num_cols), sizeof(num_cols));

    // Convert from big-endian to host endian / 从大端序转换为主机字节序
    magic = swap32(magic);
    num_items = swap32(num_items);
    num_rows = swap32(num_rows);
    num_cols = swap32(num_cols);

    // Verify image file format / 验证图像文件格式
    if (magic != 0x803) {
        std::cerr << "Invalid image file format" << std::endl;
        return false;
    }

    // Read label file header / 读取标签文件头
    uint32_t label_magic, num_labels;
    label_file.read(reinterpret_cast<char*>(&label_magic), sizeof(label_magic));
    label_file.read(reinterpret_cast<char*>(&num_labels), sizeof(num_labels));

    // Convert label file header / 转换标签文件头
    label_magic = swap32(label_magic);
    num_labels = swap32(num_labels);

    // Verify label file format / 验证标签文件格式
    if (label_magic != 0x801) {
        std::cerr << "Invalid label file format" << std::endl;
        return false;
    }

    // Check consistency between images and labels / 检查图像和标签数量是否一致
    if (num_items != num_labels) {
        std::cerr << "Number of images doesn't match number of labels" << std::endl;
        return false;
    }

    // Prepare storage / 准备存储空间
    samples.resize(num_items);
    std::vector<unsigned char> pixels(num_rows * num_cols);

    // Read and process each sample / 读取并处理每个样本
    for (uint32_t i = 0; i < num_items; ++i) {
        // Read label / 读取标签
        unsigned char label;
        label_file.read(reinterpret_cast<char*>(&label), 1);
        samples[i].label = static_cast<int>(label);

        // Read image / 读取图像
        image_file.read(reinterpret_cast<char*>(pixels.data()), pixels.size());

        // Normalize pixel values to [0,1] / 将像素值归一化到[0,1]范围
        for (size_t j = 0; j < pixels.size(); ++j) {
            samples[i].image[j] = static_cast<float>(pixels[j]) / 255.0f;
        }

        // Show progress / 显示进度
        if (i % 1000 == 0) {
            std::cout << "\rLoading data: " << (i * 100.0f / num_items) << "%" << std::flush;
        }
    }
    std::cout << "\rLoading data: 100%" << std::endl;

    return true;
}

int main() {
    // Timing events
    cudaEvent_t total_start, total_stop;
    cudaEventCreate(&total_start);
    cudaEventCreate(&total_stop);
    cudaEventRecord(total_start);

    cudaEvent_t load_start, load_stop;
    cudaEventCreate(&load_start);
    cudaEventCreate(&load_stop);
    cudaEventRecord(load_start);

    // Load data
    std::vector<TrainingSample> train_samples;
    std::vector<TrainingSample> test_samples;

    if (!loadMNISTImages("./data/MNIST/raw/train-images-idx3-ubyte",
                     "./data/MNIST/raw/train-labels-idx1-ubyte",
                     train_samples)) {
        return -1;
    }
    if (!loadMNISTImages("./data/MNIST/raw/t10k-images-idx3-ubyte",
                        "./data/MNIST/raw/t10k-labels-idx1-ubyte",
                        test_samples)) {
        return -1;
    }

    // 限制样本数量以适应 T4 内存
    const int max_test_samples = 1000;
    const int max_train_samples = max_test_samples * 10;

    if (test_samples.size() > max_test_samples) {
        test_samples.resize(max_test_samples);
    }
    if (train_samples.size() > max_train_samples) {
        train_samples.resize(max_train_samples);
    }

    cudaEventRecord(load_stop);
    cudaEventSynchronize(load_stop);
    float load_time;
    cudaEventElapsedTime(&load_time, load_start, load_stop);

    int num_trainsamples = train_samples.size();
    int num_testsamples = test_samples.size();

    std::cout << "Training samples: " << num_trainsamples << std::endl;
    std::cout << "Test samples: " << num_testsamples << std::endl;

    // Allocate host memory for training data
    float* h_train_images = new float[num_trainsamples * IMAGESIZE];
    int* h_train_labels = new int[num_trainsamples];

    for (int i = 0; i < num_trainsamples; ++i) {
        h_train_labels[i] = train_samples[i].label;
        std::memcpy(&h_train_images[i * IMAGESIZE], train_samples[i].image, sizeof(float) * IMAGESIZE);
    }

    // Pre-allocate all GPU memory we'll need
    float* d_train_images;
    int* d_train_labels;
    float* d_test_image;
    float* d_distances;
    int* d_sort_labels;

    // Calculate padded size once
    int pow2_size = 1;
    while (pow2_size < num_trainsamples) pow2_size <<= 1;
    int padded_size = pow2_size;

    // Allocate all GPU memory at once
    cudaMalloc(&d_train_images, num_trainsamples * IMAGESIZE * sizeof(float));
    cudaMalloc(&d_train_labels, num_trainsamples * sizeof(int));
    cudaMalloc(&d_test_image, IMAGESIZE * sizeof(float));
    cudaMalloc(&d_distances, padded_size * sizeof(float));
    cudaMalloc(&d_sort_labels, padded_size * sizeof(int));

    // Copy training data to GPU
    cudaMemcpy(d_train_images, h_train_images, num_trainsamples * IMAGESIZE * sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(d_train_labels, h_train_labels, num_trainsamples * sizeof(int), cudaMemcpyHostToDevice);

    // Pre-allocate host memory for results
    float* h_distances = new float[padded_size];
    int* h_labels = new int[padded_size];

    // Initialize padding arrays once
    if (padded_size > num_trainsamples) {
        float* h_pad_distances = new float[padded_size - num_trainsamples];
        int* h_pad_labels = new int[padded_size - num_trainsamples];
        for (int i = 0; i < padded_size - num_trainsamples; ++i) {
            h_pad_distances[i] = FLT_MAX;
            h_pad_labels[i] = -1;
        }
        cudaMemcpy(d_distances + num_trainsamples, h_pad_distances,
                  (padded_size - num_trainsamples) * sizeof(float), cudaMemcpyHostToDevice);
        cudaMemcpy(d_sort_labels + num_trainsamples, h_pad_labels,
                  (padded_size - num_trainsamples) * sizeof(int), cudaMemcpyHostToDevice);
        delete[] h_pad_distances;
        delete[] h_pad_labels;
    }

    // Start prediction timing
    cudaEvent_t predict_start, predict_stop;
    cudaEventCreate(&predict_start);
    cudaEventCreate(&predict_stop);
    cudaEventRecord(predict_start);

    // KNN parameters
    int k = 10;
    int correct_predictions = 0;

    // Process each test sample
    for (int t = 0; t < num_testsamples; ++t) {
        int test_label = test_samples[t].label;

        // Copy test image to GPU
        cudaMemcpy(d_test_image, test_samples[t].image, IMAGESIZE * sizeof(float), cudaMemcpyHostToDevice);

        int threadsPerBlock = TILE_WIDTH;
        int blocksPerGrid = (num_trainsamples + threadsPerBlock - 1) / threadsPerBlock;

        // Compute distances
        computeEuclideanDistances<<<blocksPerGrid, threadsPerBlock>>>(
            d_train_images, d_test_image, d_distances, d_sort_labels, d_train_labels, num_trainsamples
        );

        // Sort distances
        bitonicSort(d_distances, d_sort_labels, num_trainsamples);

        // Copy top k results
        cudaMemcpy(h_distances, d_distances, k * sizeof(float), cudaMemcpyDeviceToHost);
        cudaMemcpy(h_labels, d_sort_labels, k * sizeof(int), cudaMemcpyDeviceToHost);

        // 标签选择逻辑
        std::vector<float> labelCount(10, 0.0f);
        for (int i = 0; i < k; ++i) {
            float weight = 1.0f / (h_distances[i] + 1e-5);
            labelCount[h_labels[i]] += weight;
        }

        int predictedLabel = std::distance(labelCount.begin(),
            std::max_element(labelCount.begin(), labelCount.end()));

        if (predictedLabel == test_label) {
            correct_predictions++;
        }

        if (t % 100 == 0) {
            std::cout << "Processed " << t << " test samples" << std::endl;
        }
    }

    // Record prediction time
    cudaEventRecord(predict_stop);
    cudaEventSynchronize(predict_stop);
    float predict_time;
    cudaEventElapsedTime(&predict_time, predict_start, predict_stop);

    // Record total time
    cudaEventRecord(total_stop);
    cudaEventSynchronize(total_stop);
    float total_time;
    cudaEventElapsedTime(&total_time, total_start, total_stop);

    // Calculate and display results
    float accuracy = (float)correct_predictions / num_testsamples * 100.0f;

    printf("Total test samples: %d\n", num_testsamples);
    printf("Correct predictions: %d\n", correct_predictions);
    printf("Accuracy: %.2f%%\n", accuracy);
    printf("Total execution time: %.4f seconds\n", total_time/1000);
    printf("Prediction time: %.4f seconds\n", predict_time/1000);

    // Cleanup
    cudaEventDestroy(total_start);
    cudaEventDestroy(total_stop);
    cudaEventDestroy(load_start);
    cudaEventDestroy(load_stop);
    cudaEventDestroy(predict_start);
    cudaEventDestroy(predict_stop);

    // Free memory
    delete[] h_train_images;
    delete[] h_train_labels;
    delete[] h_distances;
    delete[] h_labels;
    cudaFree(d_train_images);
    cudaFree(d_train_labels);
    cudaFree(d_test_image);
    cudaFree(d_distances);
    cudaFree(d_sort_labels);

    return 0;
}

In [43]:
%cuda_group_run --group "knn" --compiler-args "-O3 -g -std=c++20 -arch=sm_75"

Loading data: 100%
Loading data: 100%
Training samples: 10000
Test samples: 1000
Processed 0 test samples
Processed 100 test samples
Processed 200 test samples
Processed 300 test samples
Processed 400 test samples
Processed 500 test samples
Processed 600 test samples
Processed 700 test samples
Processed 800 test samples
Processed 900 test samples
Total test samples: 1000
Correct predictions: 477
Accuracy: 47.70%
Total execution time: 0.8644 seconds
Prediction time: 0.6952 seconds

